# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 4


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 935
po2 935
albumin 558
hba1c 444
lymphocyte 585
height 415
urine-ketone 343
crp 57


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,rr,saps2,sbp,scr,sodium,sofa,use_NaHCO3,uti,wbc,weight
0,13616762,20066510,31257435,True,NaN,64,NaN,NaN,NaN,NaN,...,NaN,30,NaN,NaN,NaN,3,0.0,0,NaN,130.0
0,17922352,20070680,39895564,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,26,NaN,NaN,NaN,5,0.0,0,NaN,NaN
0,11236474,20146147,34965843,True,NaN,40,NaN,NaN,NaN,NaN,...,NaN,21,NaN,NaN,NaN,3,0.0,1,NaN,NaN
0,18135181,20173681,30217596,True,NaN,54,NaN,NaN,NaN,NaN,...,NaN,24,NaN,NaN,NaN,1,0.0,0,NaN,103.5
0,12851923,20235113,35081736,False,NaN,84,NaN,NaN,NaN,NaN,...,NaN,42,NaN,NaN,NaN,5,0.0,1,NaN,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,11084536,29816115,31328473,False,NaN,25,NaN,NaN,NaN,NaN,...,NaN,17,NaN,NaN,NaN,1,0.0,0,NaN,NaN
0,11166681,29877040,38192611,True,NaN,69,NaN,NaN,NaN,NaN,...,NaN,29,NaN,NaN,NaN,4,0.0,0,NaN,NaN
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,NaN,32,NaN,NaN,NaN,3,0.0,0,NaN,47.4
0,18624988,29915993,31559071,True,NaN,82,NaN,NaN,NaN,NaN,...,NaN,63,NaN,NaN,NaN,11,0.0,0,NaN,120.3


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

11816198

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

11410224

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-20 15:52:54.102342: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-20 15:52:54.593476: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


2024-06-20 15:52:54.991829: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 15:52:54.995275: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an 

 1/25 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - AUC: 0.4266 - accuracy: 0.5000 - loss: 0.6945 - precision: 0.5500 - recall: 0.6111

15/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.5715 - accuracy: 0.5454 - loss: 0.6867 - precision: 0.5271 - recall: 0.6697

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - AUC: 0.6032 - accuracy: 0.5660 - loss: 0.6808 - precision: 0.5513 - recall: 0.6743 - val_AUC: 0.7034 - val_accuracy: 0.6340 - val_loss: 0.6476 - val_precision: 0.6300 - val_recall: 0.6495


...

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.9182 - accuracy: 0.8360 - loss: 0.3699 - precision: 0.8724 - recall: 0.8103 - val_AUC: 0.7012 - val_accuracy: 0.6598 - val_loss: 0.8130 - val_precision: 0.6596 - val_recall: 0.6458


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - AUC: 0.5298 - accuracy: 0.5312 - loss: 0.7570 - precision: 0.5882 - recall: 0.5556

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6902 - accuracy: 0.6554 - loss: 0.6380 - precision: 0.7013 - recall: 0.6085 


Loses:

 [0.5842222571372986, 0.5766322016716003, 0.523870587348938, 0.6166512370109558, 0.6232700943946838] 0.5849292755126954 0.03542260843987992
AUCs: [0.770342230796814, 0.7769010066986084, 0.8188153505325317, 0.7223019599914551, 0.7113238573074341] 0.7599368810653686 0.03909562735513756
Accuracies: [0.6818181872367859, 0.7066115736961365, 0.7520661354064941, 0.6570248007774353, 0.6597510576248169] 0.6914543509483337 0.03517635057885373
Precisions: [0.688524603843689, 0.7131147384643555, 0.7692307829856873, 0.6363636255264282, 0.6886792182922363] 0.6991825938224793 0.043060399173868086
Recals: [0.6829268336296082, 0.707317054271698, 0.7317073345184326, 0.7459016442298889, 0.5983606576919556] 0.6932427048683166 0.052076096274059576


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - AUC: 0.5020 - accuracy: 0.4688 - loss: 0.6925 - precision: 0.5455 - recall: 0.3333

16/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.5996 - accuracy: 0.5652 - loss: 0.6813 - precision: 0.5823 - recall: 0.5626

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - AUC: 0.6285 - accuracy: 0.5951 - loss: 0.6756 - precision: 0.6067 - recall: 0.5974 - val_AUC: 0.7769 - val_accuracy: 0.7098 - val_loss: 0.6168 - val_precision: 0.7333 - val_recall: 0.6735


Epoch 2/50


...

16/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.9385 - accuracy: 0.8546 - loss: 0.3174 - precision: 0.8863 - recall: 0.8143 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.9355 - accuracy: 0.8515 - loss: 0.3237 - precision: 0.8736 - recall: 0.8266 - val_AUC: 0.7262 - val_accuracy: 0.6632 - val_loss: 0.7885 - val_precision: 0.6854 - val_recall: 0.6224


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - AUC: 0.4901 - accuracy: 0.5312 - loss: 0.8461 - precision: 0.5882 - recall: 0.5556

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6818 - accuracy: 0.6639 - loss: 0.6644 - precision: 0.7092 - recall: 0.6205 


Loses: [0.5924386382102966, 0.5771423578262329, 0.5517382621765137, 0.6061063408851624, 0.6335574388504028] 0.5921966075897217 0.027450922648472407
AUCs: [0.7631003856658936, 0.7661747932434082, 0.8055270910263062, 0.7389002442359924, 0.713528037071228] 0.7574461102485657 0.030635560212391107
Accuracies: [0.6694214940071106, 0.6983470916748047, 0.7396694421768188, 0.6859503984451294, 0.6763485670089722] 0.6939473986625672 0.024839699873142104
Precisions: [0.6423841118812561, 0.7016128897666931, 0.7459016442298889, 0.688524603843689, 0.7037037014961243] 0.6964253902435302 0.03319839112752255
Recals: [0.7886179089546204, 0.707317054271698, 0.7398374080657959, 0.688524603843689, 0.6229507923126221] 0.7094495534896851 0.0549827008585973
